In [ ]:
%%capture
!git clone https://github.com/DuyMinhLe13/Double-Agent-Tetris
!pip install pygame
!git clone https://github.com/DuyMinhLe13/notebook-video-writer
import os
import shutil
shutil.move('/kaggle/working/Double-Agent-Tetris/TetrisBattle', '/kaggle/working')
shutil.move('/kaggle/working/Double-Agent-Tetris/CustomAgent.py', '/kaggle/working')
shutil.move('notebook-video-writer/notebook_video_writer', '/kaggle/working')

In [ ]:
!pip install moviepy

In [ ]:
import os
os.environ['XDG_RUNTIME_DIR'] = '/tmp'

In [ ]:
from TetrisBattle.envs.tetris_env import TetrisDoubleEnv, TetrisSingleEnv
from TetrisBattle.envs.tetris_env import TetrisSingleInterface
import numpy as np
#from notebook_video_writer import VideoWriter

In [ ]:
import numpy as np
import copy

GRID_WIDTH = 10
GRID_DEPTH = 20

BLOCK_LENGTH = 4
BLOCK_WIDTH = 4

ipieces = [[[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0]],
          [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 1, 1], [0, 0, 0, 0]],
          [[0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0]],
          [[0, 0, 0, 0], [1, 1, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
opieces = [[[0, 0, 0, 0], [0, 2, 2, 0], [0, 2, 2, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 2, 2, 0], [0, 2, 2, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 2, 2, 0], [0, 2, 2, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 2, 2, 0], [0, 2, 2, 0], [0, 0, 0, 0]]]
jpieces = [[[0, 3, 3, 0], [0, 0, 3, 0], [0, 0, 3, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 3, 3, 3], [0, 3, 0, 0], [0, 0, 0, 0]],
          [[0, 0, 3, 0], [0, 0, 3, 0], [0, 0, 3, 3], [0, 0, 0, 0]],
          [[0, 0, 0, 3], [0, 3, 3, 3], [0, 0, 0, 0], [0, 0, 0, 0]]]
lpieces = [[[0, 0, 4, 0], [0, 0, 4, 0], [0, 4, 4, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 4, 4, 4], [0, 0, 0, 4], [0, 0, 0, 0]],
          [[0, 0, 4, 4], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 0, 0]],
          [[0, 4, 0, 0], [0, 4, 4, 4], [0, 0, 0, 0], [0, 0, 0, 0]]]
zpieces = [[[0, 5, 0, 0], [0, 5, 5, 0], [0, 0, 5, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 5, 5, 0], [5, 5, 0, 0], [0, 0, 0, 0]],
          [[0, 5, 0, 0], [0, 5, 5, 0], [0, 0, 5, 0], [0, 0, 0, 0]],
          [[0, 0, 5, 5], [0, 5, 5, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]
spieces = [[[0, 0, 6, 0], [0, 6, 6, 0], [0, 6, 0, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 6, 6, 0], [0, 0, 6, 6], [0, 0, 0, 0]],
          [[0, 0, 6, 0], [0, 6, 6, 0], [0, 6, 0, 0], [0, 0, 0, 0]],
          [[6, 6, 0, 0], [0, 6, 6, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]
tpieces = [[[0, 0, 7, 0], [0, 7, 7, 0], [0, 0, 7, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 7, 7, 7], [0, 0, 7, 0], [0, 0, 0, 0]],
          [[0, 0, 7, 0], [0, 0, 7, 7], [0, 0, 7, 0], [0, 0, 0, 0]],
          [[0, 0, 7, 0], [0, 7, 7, 7], [0, 0, 0, 0], [0, 0, 0, 0]]]
lspieces = [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] #this is the lines sent piece aka garbage lines

PIECES_DICT = {
    'I': ipieces, 'O': opieces, 'J': jpieces,
    'L': lpieces, 'Z': zpieces, 'S': spieces,
    'T': tpieces, 'G': lspieces
}

PIECE_NUM2TYPE = {1: 'I', 2: 'O', 3: 'J', 4: 'L', 5: 'Z', 6: 'S', 7: 'T', 8: 'G'}
PIECE_TYPE2NUM = {val: key for key, val in PIECE_NUM2TYPE.items()}
POSSIBLE_KEYS = ['I', 'O', 'J', 'L', 'Z', 'S', 'T']

def heightForColumn(grid, column):
    height, width = grid.shape
    for i in range(0, height):
        if grid[i][column] != 0:
            return height-i
    return 0

def heights(grid):
    result = []
    height, width = grid.shape
    for i in range(0, width):
        result.append(heightForColumn(grid, i))
    return result

def numberOfHoleInColumn(grid, column):
    result = 0
    maxHeight = heightForColumn(column)
    for height, line in enumerate(reversed(grid)):
        if height > maxHeight: break
        if line[column] == 0 and height < maxHeight:
            result+=1
    return result

def numberOfHoleInRow(grid, line):
    result = 0
    height, width = grid.shape
    for index, value in enumerate(grid[height-1-line]):
        if value == 0 and heightForColumn(index) > line:
            result += 1
    return result

def heuristic_hold_I_for_tetris(grid, hold_piece, next_piece):  #  hold_piece là list các mảng xoay
    """
    Heuristic khuyến khích hold khối I để ăn Tetris.
    """
    hold_piece_id = 0 # khởi tạo id khối hold
    if hold_piece != 0: # Nếu hold_piece không rỗng (không phải là 0)
        for row in hold_piece[0]:  # Lấy mảng xoay đầu tiên
            for cell in row:  # Tìm giá trị đầu tiên khác 0 trong mảng
                if cell != 0:
                    hold_piece_id = cell  # Lấy ID của khối
                    break
            if hold_piece_id != 0:
                break

    if PIECE_NUM2TYPE.get(hold_piece_id) == 'I':  # Sử dụng hold_piece_id (int) làm key
        lines_cleared = lines_cleared_potential(grid, hold_piece)
        if lines_cleared >= 4:
            return 1  # Thưởng điểm cao nếu có thể ăn Tetris
        
    elif next_piece != 0 and next_piece[0][0][0] == 1:
        return 0.5
    else:
        return 0

def flatten_list(nested_list):
    """Làm phẳng một list phức tạp thành một list 1 chiều chứa các số."""
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))  # Gọi đệ quy nếu gặp list
        elif isinstance(item, (int, float, np.integer, np.floating)): #nếu item là số thì append vào
            flat_list.append(item)
        elif isinstance(item, np.ndarray): #nếu là mảng numpy thì convert sang list và extend vào
            flat_list.extend(item.flatten().tolist())
    return flat_list

def heuristics_for_oppo(grid):
    columns_heights = heights(grid)
    return columns_heights + [aggregateHeight(columns_heights)] + numberOfHoles(grid, columns_heights) + bumpinesses(columns_heights) + [completLine(grid), maxPitDepth(columns_heights), maxHeightColumns(columns_heights), minHeightColumns(columns_heights)]

def heuristics(grid, hold_piece, next_piece):
    columns_heights = heights(grid)
    
    return flatten_list(columns_heights + [aggregateHeight(columns_heights)] + numberOfHoles(grid, columns_heights) + bumpinesses(columns_heights) + [completLine(grid), maxPitDepth(columns_heights), maxHeightColumns(columns_heights), minHeightColumns(columns_heights), heuristic_hold_I_for_tetris(grid, hold_piece, next_piece)])

def aggregateHeight(heights):
    result = sum(heights)
    return result

def completLine(grid):
    result = 0
    height, width = grid.shape
    for i in range (0, height) :
        if 0 not in grid[i]:
            result+=1
    return result

def bumpinesses(heights):
    result = []
    for i in range(0, len(heights)-1):
        result.append(abs(heights[i]-heights[i+1]))
    return result

def numberOfHoles(grid, heights):
    results = []
    height, width = grid.shape
    for j in range(0, width) :
        result = 0
        for i in range (0, height) :
            if grid[i][j] == 0 and height-i < heights[j]:
                result+=1
        results.append(result)
    return results

def maxHeightColumns(heights):
    return max(heights)

def minHeightColumns(heights):
    return min(heights)

def maxPitDepth(heights):
    return max(heights)-min(heights)

def get_feasible(block):
    feasibles = []

    b = block

    for x in range(BLOCK_WIDTH):
        for y in range(BLOCK_LENGTH):
            if b[x][y] > 0:
                feasibles.append([x, y])

    return feasibles

def collide(grid, block, px, py):
    feasibles = get_feasible(block)

    for pos in feasibles:
        # print(px + pos[0], py + pos[1])
        if px + pos[0] > GRID_WIDTH - 1:   # right
            return True

        if px + pos[0] < 0:   # left
            return True

        if py + pos[1] > len(grid[0]) - 1:  # down
            return True

        if py + pos[1] < 0:   # up
            continue

        if grid[px + pos[0]][py + pos[1]] > 0:
            # print(px, py)
            # print(px + pos[0], py + pos[1])
            # print("Touch")
            return True

    return False

def collideDown(grid, block, px, py):
    return collide(grid, block, px, py + 1)

def hardDrop(grid, block, px, py):
    y = 0
    x = 0
    if collideDown(grid, block, px, py) == False:
        x = 1
    if x == 1:
        while True:
            py += 1
            y += 1
            if collideDown(grid, block, px, py) == True:
                break

    return y

def get_block_list_id(one_hot):
  for i in range(len(one_hot)):
    if one_hot[i] == 1:
      return i + 1
  return 0

def get_grid(px, block, state):
    return_grids = np.zeros(shape=(10, 20), dtype=np.float32)

    grid = copy.deepcopy(state[:, :10].reshape(20, 10).transpose())
    grid[grid == 0.7] = 0.0
    grid[grid == 0.3] = 0.0

    block, px, py = block, px, -2

    add_y = hardDrop(grid, block, px, py)

    for x in range(0,4):
        for y in range(0,4):
            if block[x][y] > 0:
                # draw ghost grid
                if -1 < px + x < 10 and -1 < py + y + add_y < 20:
                    grid[px + x][py + y + add_y] = 0.3

                if -1 < px + x < 10 and -1 < py + y < 20:
                    grid[px + x][py + y] = 0.7
    return grid.transpose()

def cnt_cleared_lines(state):
    state = state[:, :10].reshape(20, 10)
    state[state == 0.3] = 1.0
    state[state != 1.0] = 0.0
    cnt_full_lines = 0
    for line in state:
      check = True
      for cell in line:
        if cell == 0:
          check = False
          break
      if check == True:
        cnt_full_lines += 1

    return cnt_full_lines

def lines_for_combo(n_combos):
    if n_combos == -1:
      return -1
    elif n_combos == 0:
      return 0
    elif n_combos == 1 or n_combos == 2:
      return 1
    elif n_combos == 3 or n_combos == 4:
      return 2
    elif n_combos == 5 or n_combos == 6:
      return 3
    else:
      return 4


class Agent:
    def __init__(self, turn):
        # dir_path = os.path.dirname(os.path.realpath(__file__))
        # weight_file_path = os.path.join(dir_path, turn, 'weight')

        self.moves = []
        self.weights=[0.3231703693700171, -0.5302147889445248, 0.11075906668264722, -0.8309864290669604, -0.46260161839717095, -0.22447470130742941, 0.4643909375959479, -0.48532243395992664, -1.2189919528345148, 0.2986404976715649, -3.366022481771943, -1.5170202052807122, -0.6229115164086383, -0.8405691978047725, -0.46234856576173666, 0.5989737864945853, -0.23585077866559023, -0.5544979886079159, -1.509135254485805, -0.2596220030272241, 0.25810904529226797, -1.235583336296609, -0.196284985767132, -0.8757604472934051, -0.19556848864032964, -0.1600381924043729, 0.14558307796031955, -1.5282331815220205, -1.0895089723314406, -0.8758970221842021, 0.5232456661943335, -0.8666905685217378, -2.155881873482987, 0.5985656999013999, 0.4859977675981289, -0.7265747226666115, 0.6511813447892787, 0.5761870323553704, -0.8300319562505459, -0.2662974649793782, -1.0408113468810247, 0.783850795975288, 2.917107290392447, 0.6178467973314473, 0.5505652573043487, -0.9298952019297602, -0.5743897739438917, 0.8339642699916242, 0.47707222133512023, 0.6052428969361493, -1.6590273413960814, -0.950330264014789, -0.360606211502262, 1.618861783528971, 2.2894019641975993, -0.9796220089939183, -0.540486380165355, 0.28263791612155836, 1.643223023569673, 2.8167680702487035, -0.5806216650384408, -0.9851392180521459, 0.8989378794175819, -1.3569439828222674, 0.4948137231700634, 0.8934639555685278, 0.6473124943085833, 1.1194600259372538, 0.35805118907452027, -0.20774206480015645, 0.6265351233839038, 1.5532020095980361, 0.5590604733493032, 0.18537500273490937, -0.2516918936420716, -0.6425948054684691, -2.01170322853233, -0.7231226405899464, -2.2826117198085134, 0.2091802697307159, -0.1754158268051744, 0.7996376650848438, 0.8340327396168011, -2.7021872113839995, -1.9523949097598425, 0.5]
        
    def best_action(self, state):
        n_block_list_id = get_block_list_id(state[6, 10:17].reshape(7))
        h_block_list_id = get_block_list_id(state[0, 10:17].reshape(7))
        f_block_list_id = get_block_list_id(state[1, 10:17].reshape(7))

        bestRotation = None
        bestOffset = None
        bestScore = None
        bestSwap = None

        possible_blocks = []

        if h_block_list_id == 0:
          hold_piece = 0 #hold rỗng
          possible_blocks = [PIECES_DICT[PIECE_NUM2TYPE[n_block_list_id]], PIECES_DICT[PIECE_NUM2TYPE[f_block_list_id]]]
        else:
          hold_piece = PIECES_DICT[PIECE_NUM2TYPE[h_block_list_id]] # Lấy khối đang hold
          possible_blocks = [PIECES_DICT[PIECE_NUM2TYPE[n_block_list_id]], PIECES_DICT[PIECE_NUM2TYPE[h_block_list_id]]]

        bestScore_ = [0, 0]
        bestOffset_ = [0, 0]
        bestRotation_ = [0, 0]

        my_combo = copy.deepcopy(state[7, 11]).reshape(1)*10
        block_infos = copy.deepcopy(state[0:7, 10:17]).reshape(7,7).flatten()
        lines_about_to_get = copy.deepcopy(state[7, 13]).reshape(1)*20

        for i in range(len(possible_blocks)):

            if i == 0:  # Nếu không hold
                next_piece = PIECES_DICT[PIECE_NUM2TYPE[f_block_list_id]] #Khối tiếp theo sẽ là khối f
                if h_block_list_id == 0: # Nếu hố hold trống, thì hold_piece=0
                    hold_piece = 0  # Hold rỗng
                else: # Nếu không thì hold_piece là khối hiện tại đang được giữ
                    hold_piece = PIECES_DICT[PIECE_NUM2TYPE[h_block_list_id]]
            else:  # Nếu hold
                next_piece = PIECES_DICT[PIECE_NUM2TYPE[n_block_list_id]] #khối tiếp theo sẽ là khối hiện tại
                hold_piece = PIECES_DICT[PIECE_NUM2TYPE[n_block_list_id]]  # hold_piece là khối hiện tại
            
            for rotation in range(0, 4):
                cnt_check = np.count_nonzero(np.array(possible_blocks[i][rotation]).flatten() != 0)

                for offset in range(-3, 11):
                    changed_board = get_grid(offset, possible_blocks[i][rotation], state)
    
                    if np.count_nonzero(np.array(changed_board).flatten() == 0.3) == cnt_check:
                        changed_board[changed_board == 0.3] = 1.0
                        changed_board[changed_board != 1.0] = 0.0
    
                        heuristic = np.concatenate((heuristics(changed_board, hold_piece, next_piece),my_combo,lines_about_to_get,block_infos))
    
                        score = sum([a*b for a,b in zip(heuristic, self.weights)])
    
                        if bestScore_[i] == 0 or score > bestScore_[i]:
                            bestScore_[i] = score
                            bestOffset_[i] = offset
                            bestRotation_[i] = rotation


        if bestScore_[0] > bestScore_[1]:
          bestRotation = bestRotation_[0]
          bestOffset = bestOffset_[0]
          bestScore = bestScore_[0]
          bestSwap = 0
        else:
          bestRotation = bestRotation_[1]
          bestOffset = bestOffset_[1]
          bestScore = bestScore_[1]
          bestSwap = 1

        return bestOffset, bestRotation, bestSwap, bestScore

    def choose_action(self, state):
        if(len(self.moves) > 0):
          move =self.moves[0]
          self.moves.pop(0)
          return move

        offset, rotation, swap, _ = Agent.best_action(self, state)

        if swap == 1:
          self.moves.append(1)

        if offset != None:
          offset = offset - 4
          if rotation == 3:
            self.moves.append(4)
          else:
            for _ in range(0, rotation):
                self.moves.append(3)

          for _ in range(0, abs(offset)):
              if offset > 0:
                  self.moves.append(5)
              else:
                  self.moves.append(6)

          self.moves.append(2)

        move =self.moves[0]
        self.moves.pop(0)
        return move
        

class Agent_op:
    def __init__(self, turn):
        # dir_path = os.path.dirname(os.path.realpath(__file__))
        # weight_file_path = os.path.join(dir_path, turn, 'weight')

        self.moves = []
        self.weights = [0.32256802555577746, -0.5085594070779542, 0.009008278863447464, -0.6758382241806726, -0.43254691622702346, -0.2366230107308704, 0.4590966357875846, -0.5737720221277605, -1.1103978545338498, 0.27340290172121495, -3.41056171315169, -1.4839139013344107, 0.12031672258326076, -0.8171281035484166, -0.6430795412889315, 0.5622543755294824, -0.1392443920126013, -0.5684551473344335, -1.4880992024987938, -0.44875008654267196, 0.259801063190303, -1.0343127484715808, -0.19307492445141963, -0.8679107702355071, -0.19514420535828975, -0.1436021937607445, 0.1573103920796411, -1.532717599407211, -1.068322825831482, -0.7452755143577328, 0.4496246285247863, -0.3473408588141064, -2.1655043644789203, 0.3394471543294608, 0.3802336568122447, -0.4454064369064752, 1.2904347358871195, 0.5664954957030851, -0.8569607380672469, -0.7954794669957577, -0.9977643702723661, 0.541928374361818, 2.7559053232007362, 0.5925084065919668, 0.5752461709690733, -0.7146913077759729, -0.5658754020647635, 0.801925767933979, -0.21248020835168963, 0.6207928172295175, -1.7450170953035609, -1.8223912441304246, -0.4137546580573677, 1.514412536960759, 2.2844696058117635, -0.9036101693802795, -0.3966213572582891, 0.28354627445425656, 1.5928172793413085, 1.8197659858325734, -0.6242787477500965, -1.020226693244945, 0.9587740916620321, -1.6561039157276702, 0.7861747682431649, 0.7549042318952053, 0.5973598350667779, 1.1748443849523957, 0.0493173117804194, -0.3900671066664925, 0.6191512164458156, 1.566536690953708, 0.6455903487579286, -0.4962359917815489, -0.2576333196886974, -0.6633717424600586, -1.9646706228903685, -0.7309752021163923, -2.081639930051616, -1.658199909844026, -0.31894031061384326, 0.5788806512209099, 0.8508811521716769, -1.9665377859077995, -1.8791521085267497, 0.5]
        
    def best_action(self, state):
        n_block_list_id = get_block_list_id(state[6, 10:17].reshape(7))
        h_block_list_id = get_block_list_id(state[0, 10:17].reshape(7))
        f_block_list_id = get_block_list_id(state[1, 10:17].reshape(7))

        bestRotation = None
        bestOffset = None
        bestScore = None
        #bestSwap = 1 thì vứt block hiện tại vào hold và lấy block đầu tiên ở tương lai
        #bestSwap = 0 thì sử dụng block hiện tại
        #bestSwap = 1 thì hoán đổi block hiện tại với block trong hold
        bestSwap = None

        possible_blocks = []

        if h_block_list_id == 0:
          possible_blocks = [PIECES_DICT[PIECE_NUM2TYPE[n_block_list_id]], PIECES_DICT[PIECE_NUM2TYPE[f_block_list_id]]]
        else:
          possible_blocks = [PIECES_DICT[PIECE_NUM2TYPE[n_block_list_id]], PIECES_DICT[PIECE_NUM2TYPE[h_block_list_id]]]

        bestScore_ = [0, 0]
        bestOffset_ = [0, 0]
        bestRotation_ = [0, 0]

        for i in range(len(possible_blocks)):
          for rotation in range(0, 4):
              cnt_check = np.count_nonzero(np.array(possible_blocks[i][rotation]).flatten() != 0)

              for offset in range(-3, 11):
                changed_board = get_grid(offset, possible_blocks[i][rotation], state)

                if np.count_nonzero(np.array(changed_board).flatten() == 0.3) == cnt_check:
                    changed_board[changed_board == 0.3] = 1.0
                    changed_board[changed_board != 1.0] = 0.0

                    heuristic = heuristics_for_oppo(changed_board)

                    score = sum([a*b for a,b in zip(heuristic, self.weights)])

                    if bestScore_[i] == 0 or score > bestScore_[i]:
                        bestScore_[i] = score
                        bestOffset_[i] = offset
                        bestRotation_[i] = rotation


        if bestScore_[0] > bestScore_[1]:
          bestRotation = bestRotation_[0]
          bestOffset = bestOffset_[0]
          bestScore = bestScore_[0]
          bestSwap = 0
        else:
          bestRotation = bestRotation_[1]
          bestOffset = bestOffset_[1]
          bestScore = bestScore_[1]
          bestSwap = 1

        return bestOffset, bestRotation, bestSwap, bestScore

    def choose_action(self, state):
        if(len(self.moves) > 0):
          move =self.moves[0]
          self.moves.pop(0)
          return move

        offset, rotation, swap, _ = Agent_op.best_action(self, state)

        if swap == 1:
          self.moves.append(1)

        if offset != None:
          offset = offset - 4
          if rotation == 3:
            self.moves.append(4)
          else:
            for _ in range(0, rotation):
                self.moves.append(3)

          for _ in range(0, abs(offset)):
              if offset > 0:
                  self.moves.append(5)
              else:
                  self.moves.append(6)

          self.moves.append(2)

        move =self.moves[0]
        self.moves.pop(0)
        return move

In [ ]:
import random
import math
import statistics
import time
from operator import itemgetter
from sys import stdout
from functools import reduce

def individualFromDistribution(average, std):
    result = []
    for i in range(0, size):
        result.append(random.normalvariate(average[i], std[i]))
    return result

def generationFromDistribution(number, size, average, std):
    results = []
    for i in range(0, number):
        tmp = individualFromDistribution(average, std)
        results.append(tmp)
    return results

def selectBestIndividuals(population, scores, number):
    bests = [individual for _, individual in reversed(sorted(zip(scores, population)))][0:number]
    return bests

def computeAverage(population):
    result = list(reduce(lambda i1, i2: [a+b for a,b in zip(i1, i2)], population))
    result = list(map(lambda x: x/len(population), result))
    return result

def computeStandardDeviation(population):
    average = computeAverage(population)
    result = [[] for _ in range(0, len(population[0]))]
    for individual in population:
        for index, weight in enumerate(individual):
            result[index].append(weight)
    result = list(map(lambda weights: statistics.stdev(weights), result))
    return result

def lines_cleared_potential(grid, current_piece):
    """
    Tính toán số dòng tiềm năng có thể xóa được bằng khối hiện tại.
    """
    best_lines_cleared = 0

    #temp_grid = grid.copy() #bạn không thay đổi grid nên không cần dùng deepcopy
    for rotation in range(len(current_piece)):
        for offset in range(-3, GRID_WIDTH):  # Thử tất cả offset
            temp_grid = copy.deepcopy(grid) #sửa lại, đặt temp_grid vào trong vòng lặp for
            py = -2
            while not collide(temp_grid.T, current_piece[rotation], offset, py):
              py += 1

            # Đặt khối
            for x in range(BLOCK_WIDTH):
                for y in range(BLOCK_LENGTH):
                    if current_piece[rotation][y][x] > 0:
                        if 0 <= offset + x < GRID_WIDTH and 0 <= py + y < GRID_DEPTH:
                           temp_grid[GRID_DEPTH - 1 - (py + y)][offset + x] = current_piece[rotation][y][x]

            lines_cleared = completLine(temp_grid)
            best_lines_cleared = max(best_lines_cleared, lines_cleared)

    return best_lines_cleared

In [ ]:
from numpy.lib.function_base import average
from tqdm import tqdm

survivors_rate = 0.2
pieceLimit = 1000
number = 70
batch = 1000
size = 85
mutation_rate = 0.05

env = TetrisDoubleEnv(mode="rgb_array")
# agent = Agent(0)
agent_list = [Agent(0), Agent_op(1)]

generation = []
fitness_values = []
average = [0 for i in range(size)]
std = [1 for i in range(size)]
for indivual in generationFromDistribution(number - len(generation), size, average, std):
  generation.append(indivual)

for i in range(0, batch):
    fitness_values = []
    for individual in tqdm(generation):
        done = False
        state = env.reset()
        # env.game_interface.time = 1000000 * 3 / 16
        agent_list[0].weights = individual

        lines_cleared = 0
        n_block_fallen = 0
        infos = None
        while not done:
            img = env.render(mode='rgb_array')
            action = agent_list[env.game_interface.getCurrentPlayerID()].choose_action(state)
            if env.game_interface.getCurrentPlayerID() == 0:
              if action == 2:
                if int(state[7, 13].reshape(1)[0] * 20) == 0:
                  lines_cleared += ((cnt_cleared_lines(state) - int(state[7, 13].reshape(1)[0] * 20))**2 + lines_for_combo(int(state[7, 11].reshape(1)[0] * 10))**2)
                else:
                  if (cnt_cleared_lines(state) - 1) - int(state[7, 13].reshape(1)[0] * 20) < 0:
                    lines_cleared += (-10*((cnt_cleared_lines(state) - 1) - int(state[7, 13].reshape(1)[0] * 20))**2 + lines_for_combo(int(state[7, 11].reshape(1)[0] * 10))**2)
                  else:
                    lines_cleared += (((cnt_cleared_lines(state) - 1) - int(state[7, 13].reshape(1)[0] * 20))**2 + lines_for_combo(int(state[7, 11].reshape(1)[0] * 10))**2)

                n_block_fallen += 1
            state, reward, done, infos = env.step(action)
            env.take_turns()

        fitness_score = lines_cleared * 1000 + n_block_fallen
        if infos['winner'] == 1:
          fitness_score -= 500000 * state[19, 16].reshape(1)[0]
        fitness_values.append(fitness_score)

    rev_sort_fitness = fitness_values
    rev_sort_fitness = [score for score, individual in reversed(sorted(zip(fitness_values, generation)))]
    rev_sort_gene = [individual for score, individual in reversed(sorted(zip(fitness_values, generation)))]

    print("Generation", i + 1)
    print("Score: ", rev_sort_fitness)
    print("Weights: ", rev_sort_gene)
    new_generation = selectBestIndividuals(generation, fitness_values, int(number * survivors_rate))

    average = computeAverage(new_generation)
    std = list(map(lambda std: std, computeStandardDeviation(new_generation)))

    for indivual in generationFromDistribution(number - len(new_generation), size, average, std):
        new_generation.append(indivual)

    generation = new_generation